# "Truth"seeker Project Recreation

In [38]:
# Load the autoreload extension
%load_ext autoreload

# Set autoreload to automatically reload all modules
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import sys
from pathlib import Path

# Add the src directory to sys.path
sys.path.append(str(Path().resolve() / "src"))

# Import util
import util

In [40]:
import pandas as pd
from torch import cuda
from sklearn.model_selection import train_test_split
from datasets import Dataset

device = 'cuda' if cuda.is_available() else 'cpu'

device

'cpu'

In [41]:
DATA_PATH = "../data/truthseeker.csv"
df = pd.read_csv(DATA_PATH)
# Comment on on actual execution. For development, this small sample should run quickly on the CPU
df = df.sample(frac=0.0001,  random_state=27)
df.drop(columns=["Unnamed: 0"], inplace=True)
df = df[~df["5_label_majority_answer"].isin(["NO MAJORITY", "Unrelated"])]

clean_cols = ["statement", "tweet"]
df[clean_cols] = df[clean_cols].applymap(util.clean_text)

print(df.shape)
df.head()

(10, 8)


C:\Users\Straakh\AppData\Local\Temp\ipykernel_15704\1329652520.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[clean_cols] = df[clean_cols].applymap(util.clean_text)


,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer
58262,April Hunt,Unlike marijuana medical cannabis oil cannot g...,True,1.0,"medical canabis, cannot, high",You cannot open a medical cannabis dispensary...,Agree,Agree
40146,Louis Jacobson,There are more words in the IRS code than the...,True,1.0,"More words, IRS code, Bible",There are more words in the IRS code than the...,Mostly Disagree,Disagree
77627,Ciara O'Rourke,In 38 days Pelosi and Schiff are up for reelec...,False,0.0,"Pelosi, Schiff, reelection",All of the players in this coup need to be inv...,Agree,Agree
103400,Jon Greenberg,21 of people are having serious adverse events...,False,0.0,"21%,adverse,moderna",_AskMD Not hypothesis Judge forced FDA to FOI...,Disagree,Disagree
52117,Tom Kertscher,Says Donald Trump won Arizona.,False,0.0,trump Arizona win,Meghan you have been removed from the Republi...,Agree,Agree


In [42]:
# Combine tweet + statement + "ground truth" value for training.
df['full'] = df['target'].astype(str) + ' Statement: ' + df['statement'] + '| Tweet: ' + df['tweet']
df['full']

58262     True Statement: Unlike marijuana medical canna...
40146     True Statement:  There are more words in the I...
77627     False Statement: In 38 days Pelosi and Schiff ...
103400    False Statement: 21 of people are having serio...
52117     False Statement: Says Donald Trump won Arizona...
6117      True Statement: Murders this year have spiked ...
59604     True Statement: More people are apprehended af...
1211      True Statement: In Afghanistan over 100 billio...
55551     False Statement:  Records suggest more than 10...
69439     True Statement:  Texas has the highest rate of...
Name: full, dtype: object

In [ ]:
#CATEGORY = "5_label_majority_answer"
CATEGORY = "3_label_majority_answer"
df["labels"] = df[CATEGORY].astype("category").cat.codes
df.head(3)

,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer,full,consensus,labels
58262,April Hunt,Unlike marijuana medical cannabis oil cannot g...,True,1.0,"medical canabis, cannot, high",You cannot open a medical cannabis dispensary...,Agree,Agree,True Statement: Unlike marijuana medical canna...,True,0
40146,Louis Jacobson,There are more words in the IRS code than the...,True,1.0,"More words, IRS code, Bible",There are more words in the IRS code than the...,Mostly Disagree,Disagree,True Statement: There are more words in the I...,False,1
77627,Ciara O'Rourke,In 38 days Pelosi and Schiff are up for reelec...,False,0.0,"Pelosi, Schiff, reelection",All of the players in this coup need to be inv...,Agree,Agree,False Statement: In 38 days Pelosi and Schiff ...,False,0


In [44]:
# Split data into train (80%), validation (10%), and test (10%) splits
train_statements, temp_statements = train_test_split(df['statement'].unique(), test_size=0.2, random_state=27)
valid_statements, test_statements = train_test_split(temp_statements, test_size=0.5, random_state=27)

# Create the dataframes based on the statements
train_df = df[df['statement'].isin(train_statements)]
valid_df = df[df['statement'].isin(valid_statements)]
test_df = df[df['statement'].isin(test_statements)]

train_df.head(3)

,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer,full,consensus,labels
58262,April Hunt,Unlike marijuana medical cannabis oil cannot g...,True,1.0,"medical canabis, cannot, high",You cannot open a medical cannabis dispensary...,Agree,Agree,True Statement: Unlike marijuana medical canna...,True,0
40146,Louis Jacobson,There are more words in the IRS code than the...,True,1.0,"More words, IRS code, Bible",There are more words in the IRS code than the...,Mostly Disagree,Disagree,True Statement: There are more words in the I...,False,1
103400,Jon Greenberg,21 of people are having serious adverse events...,False,0.0,"21%,adverse,moderna",_AskMD Not hypothesis Judge forced FDA to FOI...,Disagree,Disagree,False Statement: 21 of people are having serio...,True,1


In [45]:
# Convert train and test DataFrames into Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
valid_dataset = Dataset.from_pandas(valid_df)

train_dataset = train_dataset.map(util.tokenize_function, batched=True)
test_dataset = test_dataset.map(util.tokenize_function, batched=True)
valid_dataset = valid_dataset.map(util.tokenize_function, batched=True)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [46]:
OUT = "2_class"
# OUT = "4_class"
util.train(train_dataset, test_dataset, valid_dataset, OUT)

c:\Users\Straakh\anaconda3\envs\truth\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Straakh\Documents\Git\Truthseeker Dataset\notebooks\src\util.py:130: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.616632878780365, 'eval_accuracy': 1.0, 'eval_runtime': 0.6841, 'eval_samples_per_second': 1.462, 'eval_steps_per_second': 1.462, 'epoch': 1.0}


  0%|          | 0/1 [00:00<?, ?it/s]

...

{'eval_loss': 0.6115965247154236, 'eval_accuracy': 1.0, 'eval_runtime': 0.7036, 'eval_samples_per_second': 1.421, 'eval_steps_per_second': 1.421, 'epoch': 3.0}
{'train_runtime': 41.5444, 'train_samples_per_second': 0.578, 'train_steps_per_second': 0.072, 'train_loss': 0.6942702134450277, 'epoch': 3.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.616632878780365, 'eval_accuracy': 1.0, 'eval_runtime': 0.6529, 'eval_samples_per_second': 1.532, 'eval_steps_per_second': 1.532, 'epoch': 3.0}


  0%|          | 0/1 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-0.0994429]], dtype=float32), label_ids=array([0], dtype=int64), metrics={'test_loss': 0.6446613073348999, 'test_accuracy': 1.0, 'test_runtime': 0.7368, 'test_samples_per_second': 1.357, 'test_steps_per_second': 1.357})
